In [1]:
import json

In [2]:
with open('recipes.json', encoding="utf8") as f:
    data = json.load(f)


In [3]:
from pandas.io.json import json_normalize
df = json_normalize(data[2]['data'])

In [4]:
df.head()

,id,name,ingredients,method,image,video
0,1,KALAKAND SWEET / MITHAI,Milk powder 2 cups~Sugar 1/2 cup or to taste~...,"In a heat resistant bowl add powder milk, crea...",http://www.ainycooks.com/wp-content/uploads/20...,https://www.youtube.com/embed/8ApA0f7vs00
1,2,ARIZONA CHICKEN STEAK,Chicken fillets 2 flattened~Salt to tatse~Whit...,"Marinate chicken fillet with salt, pepper, mus...",http://www.ainycooks.com/wp-content/uploads/20...,None
2,3,CHICKEN CREAMY PIE,Chicken boiled and shredded 1 cup~Onion chopp...,"In a pan add butter and onion, stir it for few...",http://www.ainycooks.com/wp-content/uploads/20...,None
3,4,2 INGREDIENTS CHOCOLATE MOUSSE,Cooking Chocolate / Milk chocolate 200g~Whipp...,Add 1/2 cup cream in chocolate and microwave i...,http://www.ainycooks.com/wp-content/uploads/20...,None
4,5,KARACHI FAMOUS STUDENT BIRYANI,Rice Basmati 1/2 kg~Chicken 1/2 kg~Red chili ...,Soak rice for 20 minutes.~Boil rice and add sa...,http://www.ainycooks.com/wp-content/uploads/20...,None


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [7]:
import re

In [14]:
datas = []
for index, row in df.iterrows():
    ings = row['ingredients'].split('~')
    data = {}
    data['recipie'] = row['name']
    data['ingredients'] = ings
    datas.append(data)

In [15]:
df = json_normalize(datas)

In [16]:
df.head()

,recipie,ingredients
0,KALAKAND SWEET / MITHAI,"[Milk powder 2 cups, Sugar 1/2 cup or to tast..."
1,ARIZONA CHICKEN STEAK,"[Chicken fillets 2 flattened, Salt to tatse, W..."
2,CHICKEN CREAMY PIE,"[Chicken boiled and shredded 1 cup, Onion cho..."
3,2 INGREDIENTS CHOCOLATE MOUSSE,"[Cooking Chocolate / Milk chocolate 200g, Whi..."
4,KARACHI FAMOUS STUDENT BIRYANI,"[Rice Basmati 1/2 kg, Chicken 1/2 kg, Red chi..."


In [21]:
new = []
for s in df['ingredients']:
    s = ' '.join(s)
    new.append(s)
df['ing'] = new
l=[]
for s in df['ing']:
    
    #Remove punctuations
    s=re.sub(r'\(.*oz.\)|tte|kg|crushed|crumbles|ground|minced|tsp|tbsp|as|required|for|powder|chopped|sliced|pinch|cup|/|ml|[0-9]',
                             '', 
                             s)
    
    #Remove Digits
    s=re.sub(r"(\d)", "", s)
    
    #Remove content inside paranthesis
    s=re.sub(r'\([^)]*\)', '', s)
    
    #Remove Brand Name
    s=re.sub(u'\w*\u2122', '', s)
    
    #Convert to lowercase
    s=s.lower()
    
    #Remove Stop Words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    s= ' '.join(filtered_sentence)
    
    l.append(s)
df['ing_mod']=l

In [22]:
df.head()

,recipie,ingredients,ing,ing_mod
0,KALAKAND SWEET / MITHAI,"[Milk powder 2 cups, Sugar 1/2 cup or to tast...",Milk powder 2 cups Sugar 1/2 cup or to taste ...,milk sugar tte cream nuts
1,ARIZONA CHICKEN STEAK,"[Chicken fillets 2 flattened, Salt to tatse, W...",Chicken fillets 2 flattened Salt to tatse Whit...,chicken fillets flaned salt tatse white pepper...
2,CHICKEN CREAMY PIE,"[Chicken boiled and shredded 1 cup, Onion cho...",Chicken boiled and shredded 1 cup Onion chopp...,chicken boiled shredded onion medium bur black...
3,2 INGREDIENTS CHOCOLATE MOUSSE,"[Cooking Chocolate / Milk chocolate 200g, Whi...",Cooking Chocolate / Milk chocolate 200g Whipp...,cooking chocolate milk chocolate g whipping cream
4,KARACHI FAMOUS STUDENT BIRYANI,"[Rice Basmati 1/2 kg, Chicken 1/2 kg, Red chi...",Rice Basmati 1/2 kg Chicken 1/2 kg Red chili ...,rice bmati chicken red chili black pepper whit...


In [52]:
labels = list()
for line in df.recipie:
    labels.append(line)
texts = list()
for line in df.ing_mod:
    texts.append( line )
lengths = list()
for text in texts:
    lengths.append( len( text.split() ) )
maxlen = max( lengths )
labels = np.array( labels )
texts = np.array( texts )

In [53]:
maxlen

102

In [54]:
len(labels)

1721

In [25]:
import tensorflow as tf

In [34]:
from sklearn import preprocessing

In [55]:
le = preprocessing.LabelEncoder()

In [56]:
le.fit(labels)

LabelEncoder()

In [57]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( texts )
tokenized_messages = tokenizer.texts_to_sequences( texts )
padded_messages = tf.keras.preprocessing.sequence.pad_sequences( tokenized_messages , maxlen )
onehot_labels = tf.keras.utils.to_categorical( list(le.transform(labels)) , num_classes=1721 )

X = padded_messages
Y = onehot_labels

print( X.shape )
print( Y.shape )
print( 'MESSAGE MAXLEN = {}'.format( maxlen ) )

output_path = 'processed_data/'
np.save( '{}x.npy'.format( output_path )  , X )
np.save( '{}y.npy'.format( output_path )  , Y )

with open( 'processed_data/word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )

print( 'Data processed.')

(1721, 102)
(1721, 1721)
MESSAGE MAXLEN = 102
Data processed.


In [58]:
with open('processed_data/vocab.txt' , 'w') as file:
    for key in tokenizer.word_index.keys():
        file.write(key+' '+str(tokenizer.word_index[key])+'\n')

In [59]:
with open('processed_data/labels.txt' , 'w') as file:
    for label in labels:
        file.write(label+'\n')

In [64]:
from tensorflow.python.keras import models , optimizers , losses ,activations
from tensorflow.python.keras.layers import *
import tensorflow as tf
import time

class Classifier (object) :

	def __init__( self , number_of_classes , maxlen ):

		
		dropout_rate = 0.5

		input_shape = ( maxlen ,  )
		target_shape = ( maxlen , 1 )

		# Note : activations.leaky_relu is a CUSTOM IMPLEMENTATION. It DOES NOT EXIST in the official TensorFlow build.

		self.model_scheme = [

			Reshape( input_shape=input_shape , target_shape=( maxlen , 1 ) ),

			Conv1D( 128, kernel_size=2 , strides=1, activation=activations. , kernel_regularizer='l1'),
			MaxPooling1D(pool_size=2 ),

			Flatten() ,

			Dense( 64 , activation=activations.leaky_relu ) ,
			BatchNormalization(),
			Dropout(dropout_rate),

			Dense( number_of_classes, activation=tf.nn.softmax )

		]

		self.__model = tf.keras.Sequential(self.model_scheme)
		self.__model.compile(
			optimizer=optimizers.Adam( lr=0.0001 ),
			loss=losses.categorical_crossentropy ,
			metrics=[ 'accuracy' ] ,
		)


	def fit(self, X, Y ,  hyperparameters  ):
		initial_time = time.time()
		self.__model.fit( X  , Y ,
						 batch_size=hyperparameters[ 'batch_size' ] ,
						 epochs=hyperparameters[ 'epochs' ] ,
						 callbacks=hyperparameters[ 'callbacks'],
						 validation_data=hyperparameters[ 'val_data' ]
						 )
		final_time = time.time()
		eta = ( final_time - initial_time )
		time_unit = 'seconds'
		if eta >= 60 :
			eta = eta / 60
			time_unit = 'minutes'
		self.__model.summary( )
		print( 'Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit ) )


	def evaluate(self , test_X , test_Y  ) :
		return self.__model.evaluate(test_X, test_Y)


	def predict(self, X  ):
		predictions = self.__model.predict( X  )
		return predictions


	def save_model(self , file_path ):
		self.__model.save(file_path )


	def load_model(self , file_path ):
		self.__model = models.load_model(file_path)


In [70]:
from tensorflow.python.keras.callbacks import TensorBoard
import numpy as np
import time

X = np.load( 'processed_data/x.npy')
Y = np.load( 'processed_data/y.npy')

print( X.shape )
print( Y.shape )

# classifier = Classifier( number_of_classes=1721 , maxlen=102 )
# #classifier.load_model( 'models/model.h5' )

# parameters = {
#     'batch_size' : 100 ,
#     'epochs' : 100 ,
#     'callbacks' : [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
#     'val_data' : ( test_X , test_Y )
# }
# classifier.fit( X , Y , parameters )
# classifier.save_model( 'models/model.h5')

# loss , accuracy = classifier.evaluate( test_X , test_Y )
# print( "Loss of {}".format( loss ) , "Accuracy of {} %".format( accuracy * 100 ) )


input_dim = X.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1721, activation='sigmoid'))


(1721, 102)
(1721, 1721)


In [71]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

history = model.fit(X, Y,
                    epochs=100,
                    verbose=True,
                    validation_data=(X, Y),
                    batch_size=10)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                1030      
_________________________________________________________________
dense_4 (Dense)              (None, 1721)              18931     
Total params: 19,961
Trainable params: 19,961
Non-trainable params: 0
_________________________________________________________________
Train on 1721 samples, validate on 1721 samples
Epoch 1/100
1721/1721 [==============================] - 1s 506us/step - loss: 0.3641 - accuracy: 0.9314 - val_loss: 0.0253 - val_accuracy: 0.9987
Epoch 2/100
1721/1721 [==============================] - 1s 309us/step - loss: 0.0216 - accuracy: 0.9991 - val_loss: 0.0179 - val_accuracy: 0.9993
Epoch 3/100
1721/1721 [==============================] - 1s 311us/step - loss: 0.0173 - accuracy: 0.9993 - val_loss: 0.0153 - val_accuracy: 0.9993
Epoch 4/100
1721/1721 [

Epoch 52/100
1721/1721 [==============================] - 1s 383us/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 53/100
1721/1721 [==============================] - 1s 375us/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 54/100
1721/1721 [==============================] - 1s 372us/step - loss: 0.0022 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 55/100
1721/1721 [==============================] - 1s 382us/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 56/100
1721/1721 [==============================] - 1s 446us/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9996
Epoch 57/100
1721/1721 [==============================] - 1s 378us/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 0.0018 - val_accuracy: 0.9995
Epoch 58/100
1721/1721 [==============================] - 1s 358us/step - loss: 0.0021 - accuracy: 0.9995 - val_

In [72]:
model.save('processed_data/model.h5')

In [82]:
keras_model_path = "processed_data/model.h5"
converter = tf.compat.v1.lite.TFLiteConverter.( keras_model_path )
converter.post_training_quantize = True
tflite_buffer = converter.convert()
open( 'processed_data/model.tflite' , 'wb' ).write( tflite_buffer )

print( 'TFLite model created.')

TypeError: __init__() missing 2 required positional arguments: 'input_tensors' and 'output_tensors'